In [1]:
import numpy as np

def divide_three_classes(img, n_classes=3):
	multiclass = np.zeros((*img.shape,n_classes), dtype=np.float32)
	
	body = np.where(img<0.4, 1., 0.)
	marker = np.where((img<0.75)&(img>=0.4), 1., 0.)
	wound = np.where(img >= 0.75, 1., 0.)

	multiclass[:,:,0] = body
	multiclass[:,:,1] = marker
	multiclass[:,:,2] = wound

	return multiclass

In [2]:
import cv2

label_3 = cv2.imread('./test_data/20250605_000003.png', cv2.IMREAD_GRAYSCALE).astype(np.float32)/255.

False

## preparing real datasets for trainin

In [1]:
import os
import random
from glob import glob

FOLDS_NUMER = 6
IMGS_X_FOLDER = 909

random.seed(346)

list_imgs = glob('../data/segmented/*.png')
random.shuffle(list_imgs)

assert len(list_imgs)//FOLDS_NUMER > IMGS_X_FOLDER


In [11]:
import pandas as pd

data = pd.DataFrame({'file' : list_imgs[:IMGS_X_FOLDER], 'folder' : [0]*IMGS_X_FOLDER})

for i in range(1,FOLDS_NUMER):
  new_rows = pd.DataFrame({'file' : list_imgs[i*IMGS_X_FOLDER:(i+1)*IMGS_X_FOLDER], 'folder' : [i]*IMGS_X_FOLDER})
  data = pd.concat([data,new_rows]).reset_index(drop=True)

data

,file,folder
0,../data/segmented/-M2Xp0A2psHObp6f7Iav.-M2XpBj...,0
1,../data/segmented/-NaQsMwhq2FAV8U5jYdh.-NaQtYF...,0
2,../data/segmented/-MN2hxShPEnW3ZwMc7YZ.-Ms4LJB...,0
3,../data/segmented/-Leujocx_cjh-Ci6MrBu.-Mkq3mZ...,0
4,../data/segmented/-MIOBNUgcr0MTphTegMf.-MIOBfY...,0
...,...,...
5449,../data/segmented/-NZz-yrFJIgMkrvvyU7k.-NZz054...,5
5450,../data/segmented/-LiwHS89iphwTuLVEkcv.-LiwHZm...,5
5451,../data/segmented/-MX_tivv7eIFmiM5It8l.-MX_trj...,5
5452,../data/segmented/-NKqt26V9R-mBxkxU81X.-NKrFM2...,5


In [20]:
import os
import cv2

import numpy as np
from tqdm.auto import tqdm



for i in range(FOLDS_NUMER):
  saving_folder = f'./train_data_real{i}/'
  os.makedirs(saving_folder, exist_ok=True)
  saving_wound = saving_folder + 'wound/'
  saving_sem_mask = saving_folder + 'semantic_masks/'
  os.makedirs(saving_wound, exist_ok=True)
  os.makedirs(saving_sem_mask, exist_ok=True)
  
  data_folder = '../data/segmented/'
  wound_mask = '../data/masks/'
  body_mask = '../data/body_mask/'
  marker_mask = '../data/marker_mask/'

  rows = (data['folder']==i)

  for file in tqdm(data[rows]['file']):
    f = file.split('/')[-1]
    img = cv2.imread(data_folder+f)
    wound = cv2.imread(wound_mask+f, cv2.IMREAD_GRAYSCALE)
    body = cv2.imread(body_mask+f, cv2.IMREAD_GRAYSCALE)
    if os.path.exists(marker_mask+f):
      marker = cv2.imread(marker_mask+f, cv2.IMREAD_GRAYSCALE)
    else:
      marker = None

    semantic = np.zeros_like(wound).astype(np.float32)
    semantic = np.where(body!=0, np.ones_like(wound)*(1/3), semantic)
    semantic = np.where(wound!=0, np.ones_like(wound), semantic)
    if marker is not None:
      semantic = np.where(marker!=0, np.ones_like(wound)*(2/3), semantic)

    semantic = (semantic*255).astype(np.uint8)

    cv2.imwrite(saving_sem_mask+f, semantic)
    cv2.imwrite(saving_wound+f, img)








  0%|          | 0/909 [00:00<?, ?it/s]

  0%|          | 0/909 [00:00<?, ?it/s]

  0%|          | 0/909 [00:00<?, ?it/s]

  0%|          | 0/909 [00:00<?, ?it/s]

  0%|          | 0/909 [00:00<?, ?it/s]

  0%|          | 0/909 [00:00<?, ?it/s]